In [10]:
from cnn_model_3 import EmotionRecognition

In [1]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
train_loader = torch.load('train_dataloader.pth')
test_loader = torch.load('test_dataloader.pth')

Using device: cuda


C:\Users\Korisnik\AppData\Local\Temp\ipykernel_5852\3366855997.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_loader = torch.load('train_dataloader.pth')
C:\Users

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmotionRecognition(nn.Module):
    def __init__(self):
        super(EmotionRecognition, self).__init__()

        # Convolutional Block 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # Convolutional Block 2
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        # Dummy input to determine flatten size after conv layers
        with torch.no_grad():
            dummy = torch.zeros(1, 3, 48, 48)
            dummy = self._forward_conv(dummy)
            self.flattened_size = dummy.view(1, -1).size(1)

        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.fc2 = nn.Linear(512, 7)

    def _forward_conv(self, x):
        # Only the convolutional part, used for shape inference and forward
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [3]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
train_loader = torch.load('train_dataloader.pth')
test_loader = torch.load('test_dataloader.pth')
model = EmotionRecognition().to(device)
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

patience = 5  # Number of epochs with no improvement before stopping
best_loss = float('inf')  # Initialize best loss to a very high value
epochs_without_improvement = 0  # Counter for patience

scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
num_epochs = 50

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}",
                               ncols=100):  # Assuming train_loader is defined
        images, labels = images.to(device), labels.to(device)

        # Zero gradients, perform a backward pass, and update weights
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update the running loss
        running_loss += loss.item() * images.size(0)

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Step the learning rate scheduler
    scheduler.step()

    # Print statistics
    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples * 100

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
    print(f"Learning Rate: {scheduler.get_last_lr()[0]:.6f}")  # Print the current learning rate


C:\Users\Korisnik\AppData\Local\Temp\ipykernel_5852\1326232747.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_loader = torch.load('train_dataloader.pth')
C:\Users

Using device: cuda


Epoch 1/50: 100%|█████████████████████████████████████████████████| 449/449 [05:34<00:00,  1.34it/s]


Epoch [1/50], Loss: 1.8646, Accuracy: 27.19%
Learning Rate: 0.001000


Epoch 2/50: 100%|█████████████████████████████████████████████████| 449/449 [01:08<00:00,  6.52it/s]


...

Epoch [48/50], Loss: 0.5399, Accuracy: 76.91%
Learning Rate: 0.000063


Epoch 49/50: 100%|████████████████████████████████████████████████| 449/449 [00:23<00:00, 19.49it/s]


Epoch [49/50], Loss: 0.5379, Accuracy: 77.24%
Learning Rate: 0.000063


Epoch 50/50: 100%|████████████████████████████████████████████████| 449/449 [00:23<00:00, 19.38it/s]

Epoch [50/50], Loss: 0.5336, Accuracy: 77.07%
Learning Rate: 0.000031


In [4]:
import torch
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Set model to evaluation mode
model.eval()

emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']


# Containers for predictions and true labels
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Print classification report
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=emotion_labels))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

Classification Report:
              precision    recall  f1-score   support

       Angry       0.48      0.48      0.48       958
     Disgust       0.00      0.00      0.00       111
        Fear       0.40      0.40      0.40      1024
       Happy       0.79      0.82      0.80      1774
         Sad       0.55      0.56      0.55      1233
    Surprise       0.44      0.47      0.46      1247
     Neutral       0.78      0.71      0.75       831

    accuracy                           0.58      7178
   macro avg       0.49      0.49      0.49      7178
weighted avg       0.58      0.58      0.58      7178

Confusion Matrix:
[[ 464    0  136   66   99  175   18]
 [  76    0    9    6    4   14    2]
 [ 139    0  411   62   93  230   89]
 [  35    0   70 1455  131   58   25]
 [  84    0   97  120  687  233   12]
 [ 147    0  187   88  222  587   16]
 [  15    0  129   54   15   28  590]]


C:\Users\Korisnik\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Korisnik\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Korisnik\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
